# Лабораторная работа №4  
## Проведение исследований со случайным лесом (Random Forest)

В лабораторной работе исследуется алгоритм **случайного леса**:
- **RandomForestClassifier** для задачи классификации (loan_status);
- **RandomForestRegressor** для задачи регрессии (song_popularity).

## Импорт библиотек

In [11]:
import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, KFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    root_mean_squared_error, mean_absolute_error, r2_score
)

RANDOM_STATE = 42

## 1. Загрузка данных и формирование выборок

- отделяется целевая переменная;
- удаляются идентификаторы (`customer_id`, `song_name`);
- задаются списки категориальных и числовых признаков;
- выполняется разбиение на train/test.

In [7]:
# Пути к данным
LOAN_PATH = os.path.join("data", "Loan_approval_data_2025.csv")
SONG_PATH = os.path.join("data", "song_data.csv")

# Загрузка датасетов
loan_df = pd.read_csv(LOAN_PATH)
song_df = pd.read_csv(SONG_PATH)

# --- Classification ---
y_cls = loan_df["loan_status"].astype(int)
X_cls = loan_df.drop(columns=["loan_status"])

if "customer_id" in X_cls.columns:
    X_cls = X_cls.drop(columns=["customer_id"])

cat_cols_cls = X_cls.select_dtypes(include=["object"]).columns.tolist()
num_cols_cls = X_cls.select_dtypes(exclude=["object"]).columns.tolist()

Xc_train, Xc_test, yc_train, yc_test = train_test_split(
    X_cls, y_cls, test_size=0.2, random_state=RANDOM_STATE, stratify=y_cls
)

# --- Regression ---
y_reg = song_df["song_popularity"].astype(float)
X_reg = song_df.drop(columns=["song_popularity"])

if "song_name" in X_reg.columns:
    X_reg = X_reg.drop(columns=["song_name"])

cat_cols_reg = X_reg.select_dtypes(include=["object"]).columns.tolist()
num_cols_reg = X_reg.select_dtypes(exclude=["object"]).columns.tolist()

Xr_train, Xr_test, yr_train, yr_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=RANDOM_STATE
)

print("Classification:", Xc_train.shape, Xc_test.shape)
print("Regression:", Xr_train.shape, Xr_test.shape)

Classification: (40000, 18) (10000, 18)
Regression: (15068, 13) (3767, 13)


## 2. Бейзлайн и оценка качества (Random Forest)

Случайный лес не требует масштабирования признаков.
Для категориальных переменных выполняется one-hot кодирование.

Бейзлайн:
- RandomForestClassifier с параметрами по умолчанию (фиксируется random_state);
- RandomForestRegressor с параметрами по умолчанию (фиксируется random_state).

Далее рассчитываются метрики качества:
- классификация: Accuracy, Precision, Recall, F1, ROC-AUC;
- регрессия: RMSE, MAE, R².

In [8]:
num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median"))
])

cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

prep_cls = ColumnTransformer([
    ("num", num_pipe, num_cols_cls),
    ("cat", cat_pipe, cat_cols_cls)
])

prep_reg = ColumnTransformer([
    ("num", num_pipe, num_cols_reg),
    ("cat", cat_pipe, cat_cols_reg)
])

baseline_cls = Pipeline([
    ("prep", prep_cls),
    ("model", RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1))
])

baseline_reg = Pipeline([
    ("prep", prep_reg),
    ("model", RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1))
])

baseline_cls.fit(Xc_train, yc_train)
baseline_reg.fit(Xr_train, yr_train)

yc_pred = baseline_cls.predict(Xc_test)
yc_proba = baseline_cls.predict_proba(Xc_test)[:, 1]

yr_pred = baseline_reg.predict(Xr_test)

baseline_cls_res = {
    "accuracy": accuracy_score(yc_test, yc_pred),
    "precision": precision_score(yc_test, yc_pred, zero_division=0),
    "recall": recall_score(yc_test, yc_pred, zero_division=0),
    "f1": f1_score(yc_test, yc_pred, zero_division=0),
    "roc_auc": roc_auc_score(yc_test, yc_proba)
}

baseline_reg_res = {
    "rmse": root_mean_squared_error(yr_test, yr_pred),
    "mae": mean_absolute_error(yr_test, yr_pred),
    "r2": r2_score(yr_test, yr_pred)
}

print("Baseline classification:", baseline_cls_res)
print("Baseline regression:", baseline_reg_res)

Baseline classification: {'accuracy': 0.9122, 'precision': 0.9161719733765066, 'recall': 0.9251589464123524, 'f1': 0.9206435285610991, 'roc_auc': np.float64(0.9747151290312477)}
Baseline regression: {'rmse': 17.20353452475776, 'mae': 12.298149638534817, 'r2': 0.3860694795460795}


## 3. Улучшение бейзлайна: гипотезы и проверка

### Гипотезы для классификации
1) Подбор `n_estimators`, `max_depth`, `min_samples_leaf` улучшит качество (особенно ROC-AUC).  
2) `class_weight='balanced'` может помочь при возможном дисбалансе классов.  
3) Подбор `max_features` влияет на разнообразие деревьев → меняет качество и переобучение.

### Гипотезы для регрессии
1) Подбор `n_estimators`, `max_depth`, `min_samples_leaf` снизит RMSE.  
2) `max_features` влияет на смещение/дисперсию ансамбля и может улучшить обобщение.

Ниже проводится GridSearchCV:
- классификация: оптимизация по ROC-AUC;
- регрессия: оптимизация по neg-RMSE.

In [9]:
# --- Tuning: Classification ---
param_grid_cls = {
    "model__n_estimators": [200],
    "model__max_depth": [None, 10],
    "model__min_samples_split": [2, 10],
    "model__min_samples_leaf": [1, 5],
    "model__max_features": ["sqrt", None],
    "model__class_weight": [None, "balanced"]
}

tuned_cls = Pipeline([
    ("prep", prep_cls),
    ("model", RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=1))
])

cv_cls = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

gs_cls = GridSearchCV(
    tuned_cls,
    param_grid=param_grid_cls,
    cv=cv_cls,
    scoring="roc_auc",
    n_jobs=-1
)
gs_cls.fit(Xc_train, yc_train)

best_cls = gs_cls.best_estimator_
print("Best cls params:", gs_cls.best_params_)
print("Best CV ROC-AUC:", gs_cls.best_score_)

yc_pred2 = best_cls.predict(Xc_test)
yc_proba2 = best_cls.predict_proba(Xc_test)[:, 1]

improved_cls_res = {
    "accuracy": accuracy_score(yc_test, yc_pred2),
    "precision": precision_score(yc_test, yc_pred2, zero_division=0),
    "recall": recall_score(yc_test, yc_pred2, zero_division=0),
    "f1": f1_score(yc_test, yc_pred2, zero_division=0),
    "roc_auc": roc_auc_score(yc_test, yc_proba2)
}
print("Improved classification:", improved_cls_res)

# --- Tuning: Regression ---
param_grid_reg = {
    "model__n_estimators": [200],
    "model__max_depth": [None, 10],
    "model__min_samples_split": [2, 10],
    "model__min_samples_leaf": [1, 5],
    "model__max_features": ["sqrt", None]
}

tuned_reg = Pipeline([
    ("prep", prep_reg),
    ("model", RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1))
])

cv_reg = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

gs_reg = GridSearchCV(
    tuned_reg,
    param_grid=param_grid_reg,
    cv=cv_reg,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)
gs_reg.fit(Xr_train, yr_train)

best_reg = gs_reg.best_estimator_
print("Best reg params:", gs_reg.best_params_)
print("Best CV -RMSE:", gs_reg.best_score_)

yr_pred2 = best_reg.predict(Xr_test)

improved_reg_res = {
    "rmse": root_mean_squared_error(yr_test, yr_pred2),
    "mae": mean_absolute_error(yr_test, yr_pred2),
    "r2": r2_score(yr_test, yr_pred2)
}
print("Improved regression:", improved_reg_res)

Best cls params: {'model__class_weight': None, 'model__max_depth': None, 'model__max_features': 'sqrt', 'model__min_samples_leaf': 1, 'model__min_samples_split': 10, 'model__n_estimators': 200}
Best CV ROC-AUC: 0.9749121037240706
Improved classification: {'accuracy': 0.9127, 'precision': 0.9166966534724721, 'recall': 0.9255222524977293, 'f1': 0.9210883123926602, 'roc_auc': np.float64(0.9747544703520615)}
Best reg params: {'model__max_depth': None, 'model__max_features': 'sqrt', 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 200}
Best CV -RMSE: -17.35700555055774
Improved regression: {'rmse': 17.04182545744198, 'mae': 12.105365241972857, 'r2': 0.3975568325074823}


## 4. Имплементация случайного леса

Реализуется упрощённый случайный лес:
- базовое дерево (самописное) + bootstrap выборка для каждого дерева;
- усреднение предсказаний (для классификации — средняя вероятность, для регрессии — среднее значение).

Из-за вычислительной сложности мой лес обучается на подвыборке данных.


## 5. Подготовка данных

In [12]:
def to_dense(X):
    return X.toarray() if sparse.issparse(X) else np.asarray(X)

def sample_indices(n, max_n=8000, random_state=42):
    if n <= max_n:
        return np.arange(n)
    r = np.random.default_rng(random_state)
    return r.choice(n, size=max_n, replace=False)

# Препроцессинг тот же, что и для sklearn
Xc_tr = prep_cls.fit_transform(Xc_train)
Xc_te = prep_cls.transform(Xc_test)
yc_tr = np.asarray(yc_train).astype(int)
yc_te = np.asarray(yc_test).astype(int)

Xr_tr = prep_reg.fit_transform(Xr_train)
Xr_te = prep_reg.transform(Xr_test)
yr_tr = np.asarray(yr_train).astype(float)
yr_te = np.asarray(yr_test).astype(float)

# Для самописных деревьев/леса лучше работать с dense
Xc_tr_arr, Xc_te_arr = to_dense(Xc_tr), to_dense(Xc_te)
Xr_tr_arr, Xr_te_arr = to_dense(Xr_tr), to_dense(Xr_te)

print("CLS:", Xc_tr_arr.shape, Xc_te_arr.shape)
print("REG:", Xr_tr_arr.shape, Xr_te_arr.shape)

# Подвыборка для ускорения
idx_c = sample_indices(Xc_tr_arr.shape[0], max_n=8000, random_state=RANDOM_STATE)
idx_r = sample_indices(Xr_tr_arr.shape[0], max_n=8000, random_state=RANDOM_STATE)

Xc_tr_s, yc_tr_s = Xc_tr_arr[idx_c], yc_tr[idx_c]
Xr_tr_s, yr_tr_s = Xr_tr_arr[idx_r], yr_tr[idx_r]

print("Subsample CLS:", Xc_tr_s.shape, "Subsample REG:", Xr_tr_s.shape)

CLS: (40000, 27) (10000, 27)
REG: (15068, 13) (3767, 13)
Subsample CLS: (8000, 27) Subsample REG: (8000, 13)


## 6. Имплементация дерева + классификация/регрессия

In [13]:
class _Node:
    __slots__ = ("feature", "threshold", "left", "right", "value")
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

class MyDecisionTreeBase:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1,
                 max_features=None, random_state=42):
        self.max_depth = max_depth
        self.min_samples_split = int(min_samples_split)
        self.min_samples_leaf = int(min_samples_leaf)
        self.max_features = max_features
        self.random_state = int(random_state)
        self.rng_ = np.random.default_rng(self.random_state)
        self.root_ = None
        self.n_features_ = None

    def _feature_subset(self, n_features):
        if self.max_features is None:
            return np.arange(n_features)
        if self.max_features == "sqrt":
            k = max(1, int(np.sqrt(n_features)))
        elif self.max_features == "log2":
            k = max(1, int(np.log2(n_features)))
        elif isinstance(self.max_features, int):
            k = max(1, min(n_features, self.max_features))
        else:
            k = n_features
        return self.rng_.choice(n_features, size=k, replace=False)

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y)
        self.n_features_ = X.shape[1]
        self.root_ = self._build(X, y, depth=0)
        return self

    def _build(self, X, y, depth):
        n = X.shape[0]
        if n < self.min_samples_split:
            return _Node(value=self._leaf_value(y))
        if self.max_depth is not None and depth >= self.max_depth:
            return _Node(value=self._leaf_value(y))
        if self._stop(y):
            return _Node(value=self._leaf_value(y))

        feat_ids = self._feature_subset(self.n_features_)
        best = self._best_split(X, y, feat_ids)
        if best is None:
            return _Node(value=self._leaf_value(y))

        f, thr = best
        mask = X[:, f] <= thr
        if mask.sum() < self.min_samples_leaf or (~mask).sum() < self.min_samples_leaf:
            return _Node(value=self._leaf_value(y))

        left = self._build(X[mask], y[mask], depth+1)
        right = self._build(X[~mask], y[~mask], depth+1)
        return _Node(feature=f, threshold=thr, left=left, right=right, value=None)

    def predict(self, X):
        X = np.asarray(X, dtype=float)
        out = np.empty(X.shape[0], dtype=float)
        for i in range(X.shape[0]):
            out[i] = self._predict_one(X[i], self.root_)
        return out

    def _predict_one(self, x, node):
        while node.value is None:
            node = node.left if x[node.feature] <= node.threshold else node.right
        return node.value

    # переопределяются
    def _leaf_value(self, y): raise NotImplementedError
    def _stop(self, y): raise NotImplementedError
    def _best_split(self, X, y, feat_ids): raise NotImplementedError


class MyDecisionTreeClassifier(MyDecisionTreeBase):
    def _leaf_value(self, y):
        vals, cnts = np.unique(y, return_counts=True)
        return int(vals[np.argmax(cnts)])

    def _stop(self, y):
        return np.unique(y).size == 1

    def _gini(self, y):
        _, cnts = np.unique(y, return_counts=True)
        p = cnts / cnts.sum()
        return 1.0 - np.sum(p*p)

    def _best_split(self, X, y, feat_ids):
        base = self._gini(y)
        best_gain, best = 0.0, None

        for f in feat_ids:
            col = X[:, f]
            uniq = np.unique(col)
            if uniq.size <= 1:
                continue

            # ограничим число порогов
            if uniq.size > 40:
                thr_list = np.quantile(uniq, np.linspace(0.05, 0.95, 40))
            else:
                thr_list = (uniq[:-1] + uniq[1:]) / 2.0

            for thr in thr_list:
                mask = col <= thr
                if mask.sum() < self.min_samples_leaf or (~mask).sum() < self.min_samples_leaf:
                    continue
                g = mask.mean()*self._gini(y[mask]) + (1-mask.mean())*self._gini(y[~mask])
                gain = base - g
                if gain > best_gain:
                    best_gain, best = gain, (f, float(thr))
        return best

    def predict_proba(self, X):
        pred = self.predict(X).astype(int)
        p1 = pred.astype(float)
        p0 = 1.0 - p1
        return np.vstack([p0, p1]).T


class MyDecisionTreeRegressor(MyDecisionTreeBase):
    def _leaf_value(self, y):
        return float(np.mean(y))

    def _stop(self, y):
        return float(np.var(y)) < 1e-12

    def _mse(self, y):
        if y.size == 0:
            return 0.0
        m = float(np.mean(y))
        return float(np.mean((y - m)**2))

    def _best_split(self, X, y, feat_ids):
        base = self._mse(y)
        best_gain, best = 0.0, None

        for f in feat_ids:
            col = X[:, f]
            uniq = np.unique(col)
            if uniq.size <= 1:
                continue

            if uniq.size > 40:
                thr_list = np.quantile(uniq, np.linspace(0.05, 0.95, 40))
            else:
                thr_list = (uniq[:-1] + uniq[1:]) / 2.0

            for thr in thr_list:
                mask = col <= thr
                if mask.sum() < self.min_samples_leaf or (~mask).sum() < self.min_samples_leaf:
                    continue
                mse_split = mask.mean()*self._mse(y[mask]) + (1-mask.mean())*self._mse(y[~mask])
                gain = base - mse_split
                if gain > best_gain:
                    best_gain, best = gain, (f, float(thr))
        return best

print("Custom trees defined.")


Custom trees defined.


## 7. Имплементация RandomForest (классификация/регрессия)

In [14]:
class MyRandomForestClassifier:
    def __init__(self, n_estimators=30, max_depth=None, min_samples_split=2, min_samples_leaf=1,
                 max_features="sqrt", bootstrap=True, random_state=42):
        self.n_estimators = int(n_estimators)
        self.max_depth = max_depth
        self.min_samples_split = int(min_samples_split)
        self.min_samples_leaf = int(min_samples_leaf)
        self.max_features = max_features
        self.bootstrap = bool(bootstrap)
        self.random_state = int(random_state)
        self.rng_ = np.random.default_rng(self.random_state)
        self.trees_ = []

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y).astype(int)
        n = X.shape[0]
        self.trees_ = []

        for i in range(self.n_estimators):
            if self.bootstrap:
                idx = self.rng_.integers(0, n, size=n)
            else:
                idx = np.arange(n)

            tree = MyDecisionTreeClassifier(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf,
                max_features=self.max_features,
                random_state=self.rng_.integers(0, 10**9)
            )
            tree.fit(X[idx], y[idx])
            self.trees_.append(tree)
        return self

    def predict_proba(self, X):
        X = np.asarray(X, dtype=float)
        proba_sum = np.zeros((X.shape[0], 2), dtype=float)
        for t in self.trees_:
            proba_sum += t.predict_proba(X)
        return proba_sum / len(self.trees_)

    def predict(self, X):
        return (self.predict_proba(X)[:, 1] >= 0.5).astype(int)


class MyRandomForestRegressor:
    def __init__(self, n_estimators=30, max_depth=None, min_samples_split=2, min_samples_leaf=1,
                 max_features="sqrt", bootstrap=True, random_state=42):
        self.n_estimators = int(n_estimators)
        self.max_depth = max_depth
        self.min_samples_split = int(min_samples_split)
        self.min_samples_leaf = int(min_samples_leaf)
        self.max_features = max_features
        self.bootstrap = bool(bootstrap)
        self.random_state = int(random_state)
        self.rng_ = np.random.default_rng(self.random_state)
        self.trees_ = []

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y).astype(float)
        n = X.shape[0]
        self.trees_ = []

        for i in range(self.n_estimators):
            if self.bootstrap:
                idx = self.rng_.integers(0, n, size=n)
            else:
                idx = np.arange(n)

            tree = MyDecisionTreeRegressor(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf,
                max_features=self.max_features,
                random_state=self.rng_.integers(0, 10**9)
            )
            tree.fit(X[idx], y[idx])
            self.trees_.append(tree)
        return self

    def predict(self, X):
        X = np.asarray(X, dtype=float)
        preds = np.zeros((X.shape[0], len(self.trees_)), dtype=float)
        for i, t in enumerate(self.trees_):
            preds[:, i] = t.predict(X)
        return preds.mean(axis=1)

print("Custom random forests defined.")

Custom random forests defined.


## 8. Обучение/оценка самописного леса (baseline)

In [15]:
# --- baseline (самописный лес) ---
my_rf_cls_base = MyRandomForestClassifier(
    n_estimators=30,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features="sqrt",
    bootstrap=True,
    random_state=RANDOM_STATE
).fit(Xc_tr_s, yc_tr_s)

yc_pred_m = my_rf_cls_base.predict(Xc_te_arr)
yc_proba_m = my_rf_cls_base.predict_proba(Xc_te_arr)[:, 1]

my_baseline_cls_res = {
    "accuracy": accuracy_score(yc_te, yc_pred_m),
    "precision": precision_score(yc_te, yc_pred_m, zero_division=0),
    "recall": recall_score(yc_te, yc_pred_m, zero_division=0),
    "f1": f1_score(yc_te, yc_pred_m, zero_division=0),
    "roc_auc": roc_auc_score(yc_te, yc_proba_m),
}
print("MyRF baseline (classification):", my_baseline_cls_res)


my_rf_reg_base = MyRandomForestRegressor(
    n_estimators=30,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features="sqrt",
    bootstrap=True,
    random_state=RANDOM_STATE
).fit(Xr_tr_s, yr_tr_s)

yr_pred_m = my_rf_reg_base.predict(Xr_te_arr)

my_baseline_reg_res = {
    "rmse": root_mean_squared_error(yr_te, yr_pred_m),
    "mae": mean_absolute_error(yr_te, yr_pred_m),
    "r2": r2_score(yr_te, yr_pred_m),
}
print("MyRF baseline (regression):   ", my_baseline_reg_res)


MyRF baseline (classification): {'accuracy': 0.8919, 'precision': 0.8916430594900849, 'recall': 0.9148047229791099, 'f1': 0.903075405720434, 'roc_auc': np.float64(0.9624247145127445)}
MyRF baseline (regression):    {'rmse': 18.606115372680954, 'mae': 13.843709449783912, 'r2': 0.2818828864609889}


## 9. Улучшенная версия моего леса

In [16]:
my_rf_cls_imp = MyRandomForestClassifier(
    n_estimators=60,
    max_depth=16,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features="sqrt",
    bootstrap=True,
    random_state=RANDOM_STATE
).fit(Xc_tr_s, yc_tr_s)

yc_pred_m2 = my_rf_cls_imp.predict(Xc_te_arr)
yc_proba_m2 = my_rf_cls_imp.predict_proba(Xc_te_arr)[:, 1]

my_improved_cls_res = {
    "accuracy": accuracy_score(yc_te, yc_pred_m2),
    "precision": precision_score(yc_te, yc_pred_m2, zero_division=0),
    "recall": recall_score(yc_te, yc_pred_m2, zero_division=0),
    "f1": f1_score(yc_te, yc_pred_m2, zero_division=0),
    "roc_auc": roc_auc_score(yc_te, yc_proba_m2),
}
print("MyRF improved (classification):", my_improved_cls_res)


my_rf_reg_imp = MyRandomForestRegressor(
    n_estimators=60,
    max_depth=16,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features="sqrt",
    bootstrap=True,
    random_state=RANDOM_STATE
).fit(Xr_tr_s, yr_tr_s)

yr_pred_m2 = my_rf_reg_imp.predict(Xr_te_arr)

my_improved_reg_res = {
    "rmse": root_mean_squared_error(yr_te, yr_pred_m2),
    "mae": mean_absolute_error(yr_te, yr_pred_m2),
    "r2": r2_score(yr_te, yr_pred_m2),
}
print("MyRF improved (regression):   ", my_improved_reg_res)


MyRF improved (classification): {'accuracy': 0.8937, 'precision': 0.9023550724637681, 'recall': 0.9048138056312444, 'f1': 0.9035827664399093, 'roc_auc': np.float64(0.9627097824911927)}
MyRF improved (regression):    {'rmse': 19.038665821202123, 'mae': 14.803103984498385, 'r2': 0.24810554989802014}


## 10. Сравнение результатов и выводы

In [17]:
cls_compare = pd.DataFrame(
    [baseline_cls_res, improved_cls_res, my_baseline_cls_res, my_improved_cls_res],
    index=["sk_baseline", "sk_improved", "my_baseline", "my_improved"]
)

reg_compare = pd.DataFrame(
    [baseline_reg_res, improved_reg_res, my_baseline_reg_res, my_improved_reg_res],
    index=["sk_baseline", "sk_improved", "my_baseline", "my_improved"]
)

display(cls_compare)
display(reg_compare)


,accuracy,precision,recall,f1,roc_auc
sk_baseline,0.9122,0.916172,0.925159,0.920644,0.974715
sk_improved,0.9127,0.916697,0.925522,0.921088,0.974754
my_baseline,0.8919,0.891643,0.914805,0.903075,0.962425
my_improved,0.8937,0.902355,0.904814,0.903583,0.962710


,rmse,mae,r2
sk_baseline,17.203535,12.298150,0.386069
sk_improved,17.041825,12.105365,0.397557
my_baseline,18.606115,13.843709,0.281883
my_improved,19.038666,14.803104,0.248106


## Выводы

В данной лабораторной работе были исследованы модели случайного леса для задач классификации и регрессии с использованием библиотечных и самописных реализаций.

### Классификация (loan_status)

Случайный лес показал высокое качество уже на этапе бейзлайна. Улучшенная модель `sk_improved` демонстрирует лишь незначительный прирост метрик по сравнению с `sk_baseline`, что говорит о близости модели к оптимальным настройкам. Значения accuracy и F1-score превышают 0.92, а ROC-AUC около 0.97, что указывает на высокую способность модели различать классы.

Самописная реализация случайного леса работает корректно и показывает близкие результаты, однако по всем метрикам уступает библиотечной версии. Улучшение самописной модели дало небольшой эффект, но полностью сравняться с `sklearn` не удалось, что объясняется отсутствием оптимизаций и более простой реализацией алгоритма.

### Регрессия (song_popularity)

Для задачи регрессии случайный лес показал хорошие результаты. Улучшенная библиотечная модель позволила снизить RMSE и повысить коэффициент детерминации R² по сравнению с бейзлайном, что говорит о более точном описании зависимостей в данных.

Самописные модели регрессии заметно уступают библиотечным: значения RMSE выше, а R² ниже. При этом улучшение самописной модели не привело к росту качества, что связано с ограниченной реализацией и высокой чувствительностью алгоритма к параметрам.

### Итог

Случайный лес является эффективным алгоритмом как для классификации, так и для регрессии. Библиотечная реализация `sklearn` обеспечивает более высокое и стабильное качество за счёт оптимизированных процедур обучения, тогда как самописные модели корректны с точки зрения логики, но предназначены в первую очередь для учебных целей.
